# 주의
시험일시:

- 2021년 10월 23일 토요일 19:00~22:00

- 문제는 당일 19:00에 eCampus에 탑재.

- 답안은 당일 22:00까지 eCapmus에 탑재.

시험비율: 30% (상황에 따라 변동 가능)

시험범위: 7주차까지 범위

시험방법: 과제 대체

- 주피터노트북으로 답안 작성하고 ipynb 파일 1본을 (코드+결과 반드시 포함) eCampus에 탑재 (zip하지 않음)

- 당일 19:00 문제를 수령하고 거의 즉시 (5분 내) gitter에 '201711111 문제 수령 및 시험 시작' 메시지를 남김.

(남지지 않으면 결석 처리)

- 시험 시작 처음 30분 동안 gitter로 질문할 수 있음 (집중시험을 위해 그 후 질문 받지 않음, 꼭 필요한 질문 예외)

- 자유롭게 시험을 치루고, 마감 시한까지 eCampus에 답안 탑재. gitter에 메시지 남길 필요 없습니다.

시험답안:

- 주피터노트북 첫 칸에 학번, 이름 기재.

- 답을 적을 때는 '문제1-1의 답' 이라고 헤딩을 적음 (못 풀었으면 '풀지 못함' 이라고 적음)

- 소스코드 + 실행 결과가 출력되어야 함. 간혹 코드만 있거나 실행결과만 있는 경우 채점이 불가능함.

- 인터넷 자료, 타인의 코드 또는 도움을 받지 않음 (위반시 학교규칙에 따라 처리)

끝

# 201511080 강주영

# 문제 1: 열린데이터 수집

서울시 열린데이터에서 검색을 하면 '서울시 우리마을가게 상권분석서비스(상권-아파트)'가 있다.

(https://data.seoul.go.kr/dataList/OA-15566/S/1/datasetView.do?tab=A)

OpenAPI를 통해서 2021년 최근 100개 데이터를 JSON 형식으로 수집하자.

1) 프로그램으로 url을 만들어 출력. 그 url을 클릭하면 데이터가 출력 되어야 한다.

2) 전체 데이터 중 1건만 출력.

3) 다음 항목을 100개 정렬해서 출력.

	TRDAR_SE_CD_NM	상권_구분_코드_명

	APT_HSMP_CO	아파트_단지_수

	PC_6_HDMIL_ABOVE_HSHLD_CO	아파트_가격_6_억_이상_세대_수

	AVRG_MKTC	아파트_평균_시가

	TRDAR_CD_NM	상권_코드_명

정렬한 출력예시:

```

골목상권	5.0	0.0	150247619.0	동남로82길

발달상권	3.0	0.0	161157476.0	당산역_2

발달상권	1.0	0.0	81750000.0	서울 성북구 안암역

```

## 문제 1-1의 답

In [1]:
import urllib.parse
import os

API_URL = "http://openapi.seoul.go.kr:8088/"
API_KEY = "7667514664616a7838317142507a51"

KEY="7667514664616a7838317142507a51"
TYPE='json'
SERVICE='InfoTrdarAptQq'
START_INDEX=str(1)
END_INDEX=str(100)
STDR_YM_CD=str(2017)
params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_YM_CD)

request_url = urllib.parse.urljoin(API_URL,params)
print(request_url)

http://openapi.seoul.go.kr:8088/7667514664616a7838317142507a51/json/InfoTrdarAptQq/1/100/2017


## 문제 1-2의 답

In [2]:
import requests

response = requests.get(request_url)

resultDataJSON=response.json()
shopList = resultDataJSON['InfoTrdarAptQq']['row']

In [3]:
import pyspark;

sparkConfig = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=sparkConfig)\
    .getOrCreate()

shopRdd=spark.sparkContext.parallelize(shopList)

shopRdd.take(1)

[{'STDR_YY_CD': '2017',
  'TRDAR_SE_CD': 'A',
  'TRDAR_SE_CD_NM': '골목상권',
  'APT_HSMP_CO': 27.0,
  'AE_66_SQMT_BELO_HSHLD_CO': 115.0,
  'AE_66_SQMT_HSHLD_CO': 28.0,
  'AE_99_SQMT_HSHLD_CO': 15.0,
  'AE_132_SQMT_HSHLD_CO': 7.0,
  'AE_165_SQMT_HSHLD_CO': 15.0,
  'PC_1_HDMIL_BELO_HSHLD_CO': 35.0,
  'PC_1_HDMIL_HSHLD_CO': 86.0,
  'PC_2_HDMIL_HSHLD_CO': 21.0,
  'PC_3_HDMIL_HSHLD_CO': 10.0,
  'PC_4_HDMIL_HSHLD_CO': 6.0,
  'PC_5_HDMIL_HSHLD_CO': 7.0,
  'PC_6_HDMIL_ABOVE_HSHLD_CO': 15.0,
  'AVRG_AE': 69.0,
  'AVRG_MKTC': 213084605.0,
  'STDR_QU_CD': '1',
  'TRDAR_CD': '1000001',
  'TRDAR_CD_NM': '계동길'}]

## 문제 1-3의 답

In [4]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, DoubleType
from pyspark.sql import functions as F

schema=StructType([
    StructField("상권_구분_코드_명", StringType(), True),
    StructField("아파트_단지_수", DoubleType(), True),
    StructField("아파트_가격_6_억_이상_세대_수", DoubleType(), True),
    StructField("아파트_평균_시가", DoubleType(), True),
    StructField("상권_코드_명", StringType(), True),
])

filtredShopRdd = shopRdd\
    .map(lambda x: (x['TRDAR_SE_CD_NM'], x['APT_HSMP_CO'], x['PC_6_HDMIL_ABOVE_HSHLD_CO'], x['AVRG_MKTC'], x['TRDAR_CD_NM']))\

shopDF=spark.createDataFrame(filtredShopRdd, schema=schema)

results = shopDF.sort(F.desc("아파트_단지_수")).collect()

for r in results:
    print(f"{r['상권_구분_코드_명']} {r['아파트_단지_수']} {r['아파트_가격_6_억_이상_세대_수']} {r['아파트_평균_시가']} {r['상권_코드_명']}")

골목상권 148.0 0.0 111780753.0 동호로11길
골목상권 124.0 0.0 187656495.0 청파로71길
골목상권 119.0 4.0 255117553.0 후암로28길
골목상권 112.0 4.0 155983878.0 신흥로20길
골목상권 99.0 0.0 150307125.0 옥인길
골목상권 91.0 11.0 216778411.0 백범로79길
골목상권 87.0 0.0 107158730.0 창신길
골목상권 84.0 0.0 112274547.0 종로65길
골목상권 82.0 0.0 197220312.0 효창원로39길
골목상권 81.0 0.0 188610780.0 청파로47나길
골목상권 72.0 4.0 367774515.0 서빙고로71길
골목상권 69.0 0.0 161041418.0 금호산길
골목상권 61.0 0.0 160087851.0 고산자로6길
골목상권 58.0 0.0 203433145.0 청파로73길
골목상권 50.0 0.0 91525333.0 통일로12길
골목상권 49.0 2.0 276979389.0 이촌로22길
골목상권 48.0 21.0 362932679.0 우사단로10길
골목상권 42.0 0.0 179327855.0 소월로20길
골목상권 41.0 6.0 185866589.0 동호로15길
골목상권 40.0 0.0 92740830.0 명륜길
골목상권 40.0 0.0 194920635.0 백석동길
골목상권 40.0 43.0 268502757.0 동호로20길
골목상권 38.0 0.0 268455531.0 두텁바위로1길
골목상권 36.0 0.0 150639395.0 금호산2길
골목상권 35.0 4.0 316030238.0 대사관로34길
골목상권 33.0 0.0 315682453.0 장문로45길
골목상권 32.0 0.0 187181615.0 청구로17길
골목상권 30.0 21.0 357900379.0 이태원로54길
골목상권 27.0 15.0 213084605.0 계동길
골목상권 27.0 0.0 132853926.0 다산로33길
골목상권 26.0 0.0 

# 문제 2: RDD를 사용해서 단어빈도 계산

다음은 일부 수정된 노래가사이다.

yesterday=[

    ["Yesterday 어제는"],

    ["All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데"],

    ["Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여"],

    ["Oh, I believe in yesterday 오, 어제가 좋았는데"],

    ["Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021"],

    ["I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어"],

    ["There's a shadow hanging over me 내 위에 그림자가 드리워져 있네"],

    ["Oh!, yesterday came suddenly 오!, 이제는 너무 갑자기 와버렸어.."],

]
1) 단어빈도를 계산해서 출력

출력 예시: [('Yesterday', 1),('어제는', 1),('All', 1),('my', 1),...]

2) 빈도순으로 출력, 빈도 1이하는 제외.

3) 소문자, 하이픈, 느낌표, 숫자를 제외하고 단어빈도 계산해서 출력 (빈도 1이하는 제외)

* 소문자: 'Yesterday', 'yesterday'를 동일한 단어로 계산

* 하이픈: '그림자가', '그-림-자-가'를 동일한 단어로 계산

* 느낌표: 'oh', 'oh!'를 동일한 단어로 계산

* 숫자: '2021'는 제외하고 계산

4) 1글자 단어 (예를 들어 'i', '내') 제외하고 단어빈도 계산 (빈도 1이하는 제외)


## 문제 2-1의 답

In [5]:
yesterday=[
    ["Yesterday 어제는"],
    ["All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데"],
    ["Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여"],
    ["Oh, I believe in yesterday 오, 어제가 좋았는데"],
    ["Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021"],
    ["I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어"],
    ["There's a shadow hanging over me 내 위에 그림자가 드리워져 있네"],
    ["Oh!, yesterday came suddenly 오!, 이제는 너무 갑자기 와버렸어.."],
]

yesterdayRdd = spark.sparkContext.parallelize(yesterday)

In [6]:
wordCount = yesterdayRdd.flatMap(lambda x: x)\
    .flatMap(lambda x: x.split(" "))\
    .map(lambda x: (x, 1))\
    .reduceByKey(lambda x, y: x+y)\
    .map(lambda x:(x[0],x[1]))\
    .sortByKey(False)\

wordCount.collect()

[('참', 1),
 ('좋았는데', 1),
 ('있는', 2),
 ('있네', 1),
 ('이제는', 2),
 ('위에', 1),
 ('와버렸어..', 1),
 ('오,', 1),
 ('오!,', 1),
 ('예전', 1),
 ('여기에', 1),
 ('어제는', 1),
 ('어제가', 1),
 ('사람이', 1),
 ('보여', 1),
 ('반도', 1),
 ('문제들이', 2),
 ('못한', 1),
 ('모습의', 1),
 ('모든', 2),
 ('멀리', 1),
 ('드리워져', 1),
 ('되어버렸어', 1),
 ('너무', 1),
 ('내', 3),
 ('난', 1),
 ('그림자가', 1),
 ('그-림-자-가', 2),
 ('그', 1),
 ('것처럼', 1),
 ('것', 1),
 ('같았는데', 1),
 ('갑자기', 2),
 ('yesterday', 2),
 ('used', 1),
 ('troubles', 1),
 ('to', 3),
 ('though', 1),
 ("they're", 1),
 ('the', 1),
 ('suddenly', 1),
 ('stay', 1),
 ('shadow', 1),
 ('seemed', 1),
 ('over', 1),
 ('not', 1),
 ('my', 1),
 ('me', 1),
 ('man', 1),
 ('looks', 1),
 ('it', 1),
 ('in', 1),
 ('here', 1),
 ('hanging', 1),
 ('half', 1),
 ('far', 1),
 ('came', 1),
 ('believe', 1),
 ('be', 1),
 ('away', 1),
 ('as', 1),
 ('a', 1),
 ('Yesterday', 1),
 ("There's", 1),
 ('Suddenly', 1),
 ('Oh,', 1),
 ('Oh!,', 1),
 ('Now', 1),
 ("I'm", 1),
 ('I', 2),
 ('All', 1),
 ('2021', 2)]

## 문제 2-2의 답

In [7]:
sortedWordCount = wordCount\
    .filter(lambda x: x[1] > 1)\
    .map(lambda x: (x[1], x[0]))\
    .sortByKey(False)
sortedWordCount.collect()

[(3, '내'),
 (3, 'to'),
 (2, '있는'),
 (2, '이제는'),
 (2, '문제들이'),
 (2, '모든'),
 (2, '그-림-자-가'),
 (2, '갑자기'),
 (2, 'yesterday'),
 (2, 'I'),
 (2, '2021')]

## 문제 2-3의 답
3) 소문자, 하이픈, 느낌표, 숫자를 제외하고 단어빈도 계산해서 출력 (빈도 1이하는 제외)

소문자: 'Yesterday', 'yesterday'를 동일한 단어로 계산

하이픈: '그림자가', '그-림-자-가'를 동일한 단어로 계산

느낌표: 'oh', 'oh!'를 동일한 단어로 계산

숫자: '2021'는 제외하고 계산

In [8]:
filtred_sorted_word_count = yesterdayRdd.flatMap(lambda x: x)\
    .flatMap(lambda x: x.split(" "))\
    .map(lambda x: x.lower())\
    .map(lambda x: x.replace('-', ''))\
    .map(lambda x: x.replace('!', ''))\
    .filter(lambda x: x != '2021')\
    .map(lambda x: (x, 1))\
    .reduceByKey(lambda x, y: x+y)\
    .map(lambda x:(x[0],x[1]))\
    .filter(lambda x: x[1] > 1)\
    .map(lambda x: (x[1], x[0]))\
    .sortByKey(False)

filtred_sorted_word_count.collect()

[(3, 'yesterday'),
 (3, 'to'),
 (3, '내'),
 (3, '그림자가'),
 (2, '모든'),
 (2, '문제들이'),
 (2, '있는'),
 (2, '이제는'),
 (2, 'oh,'),
 (2, 'i'),
 (2, '오,'),
 (2, 'suddenly'),
 (2, '갑자기')]

## 문제 2-4의 답
4) 1글자 단어 (예를 들어 'i', '내') 제외하고 단어빈도 계산 (빈도 1이하는 제외)

In [9]:
excepted_one_word_count = yesterdayRdd.flatMap(lambda x: x)\
    .flatMap(lambda x: x.split(" "))\
    .filter(lambda x: len(x) > 1)\
    .map(lambda x: (x, 1))\
    .reduceByKey(lambda x, y: x+y)\
    .map(lambda x:(x[0],x[1]))\
    .filter(lambda x: x[1] > 1)\
    .map(lambda x: (x[1], x[0]))\
    .sortByKey(False)

excepted_one_word_count.collect()

[(3, 'to'),
 (2, '모든'),
 (2, '문제들이'),
 (2, '있는'),
 (2, '이제는'),
 (2, 'yesterday'),
 (2, '갑자기'),
 (2, '그-림-자-가'),
 (2, '2021')]

## 문제 3: DataFrame을 사용해서 코로나 발생동향 분석

서울시 열린데이터에서 검색을 하면 '서울특별시 코로나19 자치구별 확진자 발생동향'이 있다.

https://data.seoul.go.kr/dataList/OA-20470/S/1/datasetView.do

데이터를 '내려받기'해서 '서울특별시\ 코로나19\ 자치구별\ 확진자\ 발생동향.json' 파일을 분석하세요.



1) 전체 건수 출력. null 값을 제외한 null 건수 출력 (null이 남아있는지 확인하기 위해)

2) (null제거하고) 종로, 마포, 용산, 합계 (종로, 마포, 용산) 10건 출력.

+------+----+-------+------+

|jongno|mapo|yongsan|   tot|

+------+----+-------+------+

|  1968|4127|   3031|9126.0|



3) (null제거하고) 종로구의 2020년 3월 데이터 출력.

+-------------+------+

|       jcg_dt|jongno|

+-------------+------+

|2020.03.31.10|    13|

...



4) (null제거하고) 년 x 월별 합계 10건 출력.

+-----+-------+-------+

|month|   2020|   2021|

+-----+-------+-------+

|   07| 1273.0|62849.0|

## 데이터 읽어오기 데이터를 '내려받기'해서 '서울특별시\ 코로나19\ 자치구별\ 확진자\ 발생동향.json' 파일을 분석하세요.

In [10]:
import urllib.parse

API_URL = "http://openapi.seoul.go.kr:8088/"
API_KEY = "7667514664616a7838317142507a51"

KEY="7667514664616a7838317142507a51"
TYPE='json'
SERVICE='TbCorona19CountStatusJCG'
START_INDEX=str(1)
END_INDEX=str(1000)
# JCG_DT=str('2021.08.01.00')
params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX)

request_url = urllib.parse.urljoin(API_URL,params)
print(request_url)

http://openapi.seoul.go.kr:8088/7667514664616a7838317142507a51/json/TbCorona19CountStatusJCG/1/1000


In [11]:
import requests

response = requests.get(request_url)

resultDataJSON=response.json()
area = resultDataJSON['TbCorona19CountStatusJCG']['row']

areaDF = spark.createDataFrame(area)
areaDF = areaDF.filter(areaDF.JCG_DT != "20")

## 문제 3-1의 답

1) 전체 건수 출력. null 값을 제외한 null 건수 출력 (null이 남아있는지 확인하기 위해)

In [12]:
# 전체건수
areaDF.count()

605

In [13]:
from pyspark.sql.functions import isnan, when, count, col
# 전체건수
areaDF.select([count(c) for c in areaDF.columns]).show()

+----------+-------------+-------------+----------------+--------------+-----------------+---------+------------+----------+-------------+--------------+-----------------+---------------+------------------+--------------+-----------------+--------------+-----------------+----------------+-------------------+-----------+--------------+-------------+----------------+---------------+------------------+-------------+-------------+----------------+-------------+----------------+---------------+------------------+-----------+--------------+------------+---------------+----------+-------------+-------------+----------------+---------------+------------------+----------------+-------------------+-------------+----------------+----------------+----------------+-------------------+----------+-------------+--------------+-----------------+
|count(DDM)|count(DDMADD)|count(DOBONG)|count(DOBONGADD)|count(DONGJAK)|count(DONGJAKADD)|count(EP)|count(EPADD)|count(ETC)|count(ETCADD)|count(GANGBUK)|count(

In [14]:
# null 값을 제외한 null 건수 출력
areaDF.na.drop().count()

605

In [15]:
# null 값을 제외한 null 건수 출력
nulldf = areaDF.select([count(when(col(c).isNull(), c)).alias(c) for c in areaDF.columns]).show()

+---+------+------+---------+-------+----------+---+-----+---+------+-------+----------+--------+-----------+-------+----------+-------+----------+---------+------------+----+-------+------+---------+--------+-----------+------+------+---------+------+---------+--------+-----------+----+-------+-----+--------+---+------+------+---------+--------+-----------+---------+------------+------+---------+---------+---------+------------+---+------+-------+----------+
|DDM|DDMADD|DOBONG|DOBONGADD|DONGJAK|DONGJAKADD| EP|EPADD|ETC|ETCADD|GANGBUK|GANGBUKADD|GANGDONG|GANGDONGADD|GANGNAM|GANGNAMADD|GANGSEO|GANGSEOADD|GEUMCHEON|GEUMCHEONADD|GURO|GUROADD|GWANAK|GWANAKADD|GWANGJIN|GWANGJINADD|JCG_DT|JONGNO|JONGNOADD|JUNGGU|JUNGGUADD|JUNGNANG|JUNGNANGADD|MAPO|MAPOADD|NOWON|NOWONADD|SDM|SDMADD|SEOCHO|SEOCHOADD|SEONGBUK|SEONGBUKADD|SEONGDONG|SEONGDONGADD|SONGPA|SONGPAADD|WORK_DTTM|YANGCHEON|YANGCHEONADD|YDP|YDPADD|YONGSAN|YONGSANADD|
+---+------+------+---------+-------+----------+---+-----+---+------+---

## 문제 3-2의 답
2) (null제거하고) 종로, 마포, 용산, 합계 (종로, 마포, 용산) 10건 출력.


+------+----+-------+------+

|jongno|mapo|yongsan|   tot|

+------+----+-------+------+

|  1968|4127|   3031|9126.0|


In [16]:
columns = areaDF.columns

areaDF.na.drop().select(areaDF.JONGNO, areaDF.MAPO, areaDF.YONGSAN, (areaDF.JONGNO + areaDF.MAPO + areaDF.YONGSAN).alias('tot')).limit(10).show()


+------+----+-------+------+
|JONGNO|MAPO|YONGSAN|   tot|
+------+----+-------+------+
|  2036|4191|   3076|9303.0|
|  2023|4176|   3064|9263.0|
|  2008|4164|   3055|9227.0|
|  1989|4157|   3049|9195.0|
|  1977|4134|   3041|9152.0|
|  1968|4127|   3031|9126.0|
|  1958|4121|   3023|9102.0|
|  1949|4112|   3006|9067.0|
|  1925|4102|   2995|9022.0|
|  1900|4082|   2976|8958.0|
+------+----+-------+------+



## 문제 3-3의 답


3) (null제거하고) 종로구의 2020년 3월 데이터 출력.

+-------------+------+

|       jcg_dt|jongno|

+-------------+------+

|2020.03.31.10|    13|

...


In [17]:
from pyspark.sql.types import DateType

areaDF.na.drop().select(areaDF.JCG_DT, areaDF.JONGNO).where(areaDF['JCG_DT'].startswith("2020.03")).show()

+-------------+------+
|       JCG_DT|JONGNO|
+-------------+------+
|2020.03.31.10|    13|
|2020.03.30.10|    13|
|2020.03.29.10|    13|
|2020.03.28.10|    13|
|2020.03.27.00|    13|
|2020.03.26.00|    13|
|2020.03.25.00|    13|
|2020.03.24.00|    12|
|2020.03.23.00|    11|
|2020.03.22.00|    11|
|2020.03.21.00|    11|
|2020.03.20.00|    11|
|2020.03.19.00|    11|
|2020.03.18.00|    11|
|2020.03.17.10|    11|
|2020.03.16.10|    11|
|2020.03.15.10|    11|
|2020.03.14.10|    11|
|2020.03.13.10|    11|
|2020.03.12.10|    11|
+-------------+------+
only showing top 20 rows



## 문제 3-4의 답



4) (null제거하고) 년 x 월별 합계 10건 출력.

+-----+-------+-------+

|month|   2020|   2021|

+-----+-------+-------+

|   07| 1273.0|62849.0|

In [18]:
from pyspark.sql.types import DateType

areaDFCasted = areaDF.withColumn("year", areaDF.JCG_DT.substr(0,4))
areaDFCasted = areaDFCasted.withColumn("month", areaDF.JCG_DT.substr(6, 2))
areaDFCasted.groupBy('month').pivot('year').count().na.fill(0).show()

+-----+----+----+
|month|2020|2021|
+-----+----+----+
|   07|  31|  31|
|   11|  30|   0|
|   01|   0|  31|
|   09|  30|  30|
|   05|  31|  32|
|   08|  31|  31|
|   03|  31|  31|
|   02|   2|  28|
|   06|  30|  30|
|   10|  31|  23|
|   12|  31|   0|
|   04|  30|  30|
+-----+----+----+

